# Homework 3 - Find the perfect place to stay in Texas!

In [1]:
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import sent_tokenize, word_tokenize
import math
import numpy as np
import scipy
import heapq

In [3]:
with open('Airbnb_Texas_Rentals.csv','r',encoding="utf8") as csv_file: #opening the csv file in read mode
    csv_reader=csv.reader(csv_file)  #csv reader allows us to iterate through its lines
    
    with open('Airbnb_Texas_Rentals.tsv','w',encoding="utf8") as new_file:  #opening a new tsv file (still empty)
        csv_writer=csv.writer(new_file,delimiter='\t')  #writer allows us to write rows, and we separate the fields in these
                                                        #rows with the tab delimiter
        for line in csv_reader: #we iterate through the reader's lines 
               csv_writer.writerow(line[1:]) #write those lines (except the first element that is the index) on the new tsv

with open('Airbnb_Texas_Rentals.tsv','r',encoding="utf8") as all_files: #now we want all the documents so we first open the big tsv
    for i, row in enumerate(all_files):  #iterating through the lanes and keeping track of the order of the operation
        if i > 0:   #skipping the first line (the header)
            if row != '\n': #the rows were separated by blank spaces in the form of '\n' so we skip them
                with open('doc\doc_' + str(int((i-(i/2)))) + '.tsv','w',encoding="utf8") as new_files: #opening a new tsv for every line, (the str(int((i-(i/2))) is to keep the number of the docs 0,1,2 ecc instead of 0, 2, 4...)
                    new_files.write(row) #writing

In [4]:
def save_dict_to_file(dic, file): #defining a function to save files right away
    f = open('{}.txt'.format(file), 'w',encoding="utf8") #open a new file in write mode (empty)
    f.write(str(dic)) #write in it what we need (it will always be a dictionary, hence 'dic')
    f.close() #closing it

In [5]:
docpaths = [f for f in listdir("doc") if isfile(join("doc", f))] #joining the paths for the tsv documents in the directory

documentlist = {} #this will be for keeping track of the words in the docs {doc_i : ['house, 'garden'.....]}
vocabulary = {} #this will be useful to keep track of the word and its id {'house':1, 'garden':2}
inverted_index = {} #this is the inv idx: {1: [doc_1, doc_2, ....], 2: [doc_2, doc_6, ....]}
indexOfWord = 0 #this is used to give the id to the words in the vocabulary
for doc in docpaths: #for every document in the list of documents...
    with open('doc\{}'.format(doc), 'r',encoding="utf8") as file: #open the doc in read mode
        lines = file.readlines() #read its lines
        if len(lines) > 0 : #leaving out all the empty lines
            document = lines[0] #its a list of only one element, so we take that as a string
            cols = document.replace('\\n',' ').split('\t') #cleaning
            
            to_tokenize = cols[4] + cols[7] #taking description and title to tokenize
            tokens = nltk.word_tokenize(to_tokenize) #tokenization
            filtered_words = [nltk.stem.PorterStemmer().stem(word) for word in tokens if word not in nltk.corpus.stopwords.words('english') and word not in ".,'()"]
            # removing stopwords, special characters, stemming
            documentlist[doc] = filtered_words #creating a document list: for every document we will have the tokenized words in title and description
            
            for w in filtered_words: 
                if w not in vocabulary: #creating the vocabulary
                    indexOfWord += 1    #assigning an index that ranges from 1 to whatever
                    vocabulary[w] = indexOfWord  #for every index we give the word
                
                if vocabulary[w] not in inverted_index: #creating the inverted index
                    temp = []  #creating a list so we can append the documents that have that word
                    temp.append(doc)  #appending the first doc
                    inverted_index[vocabulary[w]] = temp #creating the index key and giving them the value
                elif doc not in inverted_index[vocabulary[w]]: #if the key was already created
                    inverted_index[vocabulary[w]].append(doc)  

#saving the files
save_dict_to_file(inverted_index,"inverted_index")
save_dict_to_file(vocabulary,"vocabulary")
save_dict_to_file(documentlist,"documentlist")

In [6]:
query = input() #taking the user query
tokens = nltk.word_tokenize(query) #tokenizing the query
query_tokens = [nltk.stem.PorterStemmer().stem(token) for token in tokens if token not in stopwords.words('english') if token not in ".,'()"]
#removing stopwords, special characters, stemming

vocabulary = open('vocabulary.txt', 'r', encoding = 'utf-8') #opening the vocabulary in read mode
vocabulary = eval(vocabulary.read()) #if we dont do eval the file will be considered a string. eval 'understands' what type of object it is

inverted_index = open('inverted_index.txt', 'r', encoding = 'utf-8')  #same here
inverted_index = eval(inverted_index.read())
        
term_ids = [vocabulary[token] for token in query_tokens if token in vocabulary] #taking the ids of the query tokenized words
search_results = [inverted_index[i] for i in term_ids if i in inverted_index] #taking the documents in the inv idx for that id

new_list = [set(list_) for list_ in search_results] #removing duplicates and preparing for intersection
intersect = set.intersection(*new_list) #gives us the documents that have all the words of the query

col_names = ["average_rate_per_night","bedrooms_count","city","date_of_listing","description","latitude","longitude","title","url"]
#making a list for the column names
df = pd.DataFrame(columns = col_names) #creating an empty df with the list cerated before

for j, doc in enumerate(intersect): #iterating through the intersection list and keeping track of the order
    with open(r'doc\{}'.format(doc), 'r',encoding="utf-8") as file: #opening the files in read mode
        csv_reader = csv.reader(file, delimiter = '\t') #csv.reader to iterate
        
        for line in csv_reader: #iterating through the lines. 1 line = the whole document
            for i, field in enumerate(line): #field = tab separated field
                df.loc[j, col_names[i]] = field #giving to [row j ,col i] the value of the field
        
pd.set_option('max_colwidth',500) #making the rows more readabale(bigger)
display(df[['title', 'description', 'city', 'url']].style.set_table_styles([ {'selector': '.row_heading, .blank', 'props': [('display', 'none;')]}]))
#displaying the df without the index

house sun


In [14]:
#TF = how may times the word is repeated within the i-th document divided by its length (term frequency)
def tf(word, document):
    return document.count(word) / len(document) 

#this is to have the number of times the word is repeated in one document
def word_containing(word, documentlist):
    return sum(1 for document in documentlist if word in documentlist[document])

#IDF(word) = loge(Total Number Of Documents / Number Of Documents with term 'word' in it)
def idf(word, documentlist):
    return math.log(len(documentlist) / word_containing(word, documentlist))

#TFIDF = TF * IDF
def tfidf(word, document, documentlist):
    return tf(word, document) * idf(word, documentlist)

In [16]:
new_inverted_index = {} #new inverted index for the scores
with open('documentlist.txt', 'r', encoding="utf-8") as file: #opening the document list
    data = eval(file.read()) #evaluating the file type
    for key,doc in documentlist.items(): #taking the keys (doc_i) and the values 
        for w in doc: #w = word in the doc, doc = ['house', 'garden', ....]
            score = tfidf(w,doc,documentlist) #tf idf of the word in the doc
            w_index = (key,score) #tuple of key (doc_i), and score (tf/idf)
            if vocabulary[w] not in new_inverted_index: #if the id of the word is not in the new inv_idx
                temp = [] #initializing empty list
                temp.append(w_index) #appending the tuple
                new_inverted_index[vocabulary[w]] = temp #giving to the newly created key (that is the id), the list just created [(doc_i, tf/idf), (doc_i+1, tf/idf)]
            elif doc not in new_inverted_index[vocabulary[w]]: #if the doc is not already in that list: 
                new_inverted_index[vocabulary[w]].append(w_index) #append to the list the new tuple

#removing duplicates
for key in new_inverted_index:
    new_inverted_index[key] = list(set(new_inverted_index[key]))
#saving
save_dict_to_file(new_inverted_index,"inverted_index_tfidf")

KeyboardInterrupt: 

In [10]:
query = input() #input query
tokens = nltk.word_tokenize(query) #tokenization
query_tokens = [nltk.stem.PorterStemmer().stem(token) for token in tokens if token not in stopwords.words('english') if token not in ".,'()"]
#removing stopwords, special characters, stemming

tfidf_query_array = [] 
for w in query_tokens: #scores 
    score = 1/math.sqrt(len(query_tokens)) #score for the words the 1/math.sqrt(len(query_tokens)) is to normalize the scores
    tfidf_query_array.append(score) #appending to a list for new score
#print(tfidf_query_array)

garden house
[0.7071067811865475, 0.7071067811865475]


In [11]:
inverted_index_tfidf = open("inverted_index_tfidf.txt", 'r', encoding = 'utf-8') #opening the new invd_idx and evaluating 
inverted_index_tfidf = eval(inverted_index_tfidf.read())


term_ids = [vocabulary[token] for token in query_tokens if token in vocabulary] #terms ids of the words of the query
search_results = [inverted_index_tfidf[idx] for idx in term_ids if idx in inverted_index_tfidf] #list of tuples (doc_i, score) for that id

cos_arrays = {} #dictionary for the cosine sim values for the docs
for list_ in search_results: #for every list of tuple (doc_i, score) for every id
    for tuple_ in list_: #for every tuple:
        if tuple_[0] not in cos_arrays: #if the doc is not in that tuple
            temp = []
            temp.append(tuple_[1]) #append to the newly created list the tf/idf
            cos_arrays[tuple_[0]] = temp #create the key (doc_i) and give the values as the list of tf/idfs
        else:
            cos_arrays[tuple_[0]].append(tuple_[1]) #if the key is already there append the value to the list

final = [(key,cos_arrays[key]) for key in cos_arrays if len(cos_arrays[key]) == len(term_ids)] #take the documents that have all of the words
print(final)

[334, 70]
[[('doc_78.tsv', 0.08510419309248655), ('doc_128.tsv', 0.18826079078034905), ('doc_397.tsv', 0.19414394049223496), ('doc_64.tsv', 0.10020332412502449), ('doc_112.tsv', 0.18272370869857407), ('doc_301.tsv', 0.3358165457162983), ('doc_185.tsv', 0.29583838551197705), ('doc_315.tsv', 0.12425212191503038), ('doc_221.tsv', 0.042845559281044956), ('doc_121.tsv', 0.11295647446820943), ('doc_498.tsv', 0.2218787891339828), ('doc_497.tsv', 0.033581654571629836), ('doc_372.tsv', 0.2218787891339828), ('doc_218.tsv', 0.3765215815606981), ('doc_4.tsv', 0.310630304787576)], [('doc_119.tsv', 0.12942929366148997), ('doc_308.tsv', 0.2142277964052248), ('doc_349.tsv', 0.37275636574509113), ('doc_145.tsv', 0.12425212191503038), ('doc_442.tsv', 0.077657576196894), ('doc_98.tsv', 0.17750303130718625), ('doc_332.tsv', 0.09136185434928704), ('doc_363.tsv', 0.10354343492919198), ('doc_362.tsv', 0.16790827285814916), ('doc_405.tsv', 0.09861279517065902), ('doc_237.tsv', 0.2142277964052248), ('doc_233.t

In [13]:
doc_sim = {} #dic for {doc_i : similarity_score, ...}
for tuple_ in final: #take the tuple in final (the values are the list of the tf/idf scores)
    sim = 1 - (scipy.spatial.distance.cosine(np.array(tfidf_query_array) , np.array(tuple_[1]))) #use this to compute the the cos similarity
    doc_sim[tuple_[0]] =  sim #key = doc_i, value = similarity
print(doc_sim)

{'doc_397.tsv': 1.0, 'doc_221.tsv': 0.9486832980505139, 'doc_121.tsv': 1.0, 'doc_497.tsv': 0.9191450300180578}


In [15]:
heap = [(-value, key) for key, value in doc_sim.items()] #list that has a tuple ordered by the smallest to the largest
largest = heapq.nsmallest(10, heap) #taking the smallest (they will actually be the largest)
largest = [(key, -value) for value, key in largest] #
print(largest)

col_names = ["average_rate_per_night","bedrooms_count","city","date_of_listing","description","latitude","longitude","title","url", "similarity"]
df = pd.DataFrame(columns = col_names)# index = False?

for j, doc in enumerate(largest):
    with open(r'doc\{}'.format(doc[0]), 'r',encoding="utf-8") as file:
        
        csv_reader = csv.reader(file, delimiter = '\t')
        for line in csv_reader:
            for i, field in enumerate(line):
                df.loc[j, col_names[i]] = field
                df.loc[j, 'similarity'] = doc[1]
        

pd.set_option('max_colwidth',500)
display(df[['title', 'description', 'city', 'url', 'similarity']].style.set_table_styles([ {'selector': '.row_heading, .blank', 'props': [('display', 'none;')]}]))

[('doc_121.tsv', 1.0), ('doc_397.tsv', 1.0), ('doc_221.tsv', 0.9486832980505139), ('doc_497.tsv', 0.9191450300180578)]


,title,description,city,url,similarity
0,Oak Cliff Tiny House,"Warm open space guesthouse cultured as a meditation and massage studio for 20+ years. The space is filled with natural light with a vaulted, shiplap ceiling while offering a restful view onto our garden and gold fish pond. Small refrigerator, microwave, Keurig, toaster oven, towel warmer, hair dryer and bicycles to tour north Oak Cliff and the Bishop Arts District.\nHot tub included, yoga and massage services available, if desired.",Dallas,https://www.airbnb.com/rooms/8799838?location=Carrollton%2C%20TX,1
1,Comfy Private Room in Fort Worth,"Our house is the perfect place to stay in Fort Worth. Walking distance to multiple museums, UNT Health Science Center, Botanical Gardens and more. Minutes from downtown and West 7th district - ideal for vacation and business travelers alike.",Fort Worth,https://www.airbnb.com/rooms/2502098?location=Benbrook%2C%20TX,1
2,1 Bedroom in quiet neighborhood,"Private bedroom with queen size bed and private bathroom. Cable TV in room.\n\nLiving room with cable TV, dining room, kitchen, outdoor patio with grill and sitting area, free wifi, laundry room with washer and dryer\n\nBrad and I work Monday through Friday 8am-5pm and my daughter is in school. We typically leave the house around 7:30am and return around 6pm. Guests will be able to reach me by cell phone if they need anything. I will make every effort to meet guests upon arrival. \nWe do have the worlds best dog, Scout. She is beyond friendly and never meets a stranger. She will most likely lay on her back for you to rub her belly as soon as you walk in the door. We keep her clean and shaved regularly. I can't stand a house that smalls like a dog so rest assured mine does not. :-)\n\nLocated next to:\nWillow Creek Park- jogging trail, volleyball, basketball, disc golf, dog park, playground \nClose to: Historic Fort Worth Stockyards, \nFort Worth Botanic Gardens, 7th street Cultural District, Downtown Fort Worth\n\nUber is the best transportation available. If I am available I would be happy to show you around.",Saginaw,https://www.airbnb.com/rooms/8305468?location=Azle%2C%20TX,0.948683
3,Private Rm/Bath @ the Gold House!,"This unique private guest room at the Gold House offers the amenities of home in a city full of things to do! The entire downstairs living room, kitchen, and dining area is open to guests. Come on down to SA town and experience Texan hospitality for yourself!\n\nPlease note: there is a second guest room at the Gold house, so if this listing is booked, do check with me to see if the second room is available! Thanks for your interest!\n\n\nCute House:\nIt's a private room upstairs with a full bed, small desk, walk-in closet, Ironing Board, and en suite bathroom. The window has a nice view of the back yard and wooded easement. The back yard is large with a little garden in the back. The kitchen was recently renovated with nice granite counter tops and new appliances. Also, there is a brand new running/hiking trail one mile away. Parking is safe and free in the form of my driveway.\n\nMore than Two People? No problem:\nThere is a second guestroom in the loft with a full bed that can be reserved via my other listing. http://airbnb.com/rooms/826725\n\nBreakfast/Snacks:\nThe living room/dining room/patio/kitchen are all fair game for guests. Breakfast is provided by way of oatmeal and any items found on the counter (Hot tea and coffee too). \n\nFour-Legged Friends: \nI do have two wonderful dogs (lab and husky), but don't worry they aren't allowed upstairs. They stay outside during the day, and sleep in their beds downstairs at night. They are both full of sugar - but if you have a fear of dogs then this place is not for you(URL HIDDEN)\nLocation:\nIt is literally right next to a brand new grocery store but you wouldn't know it from the back yard. It is 10-15 minutes from La Cantera - the gorgeous outdoor shopping mall. As well as Six Flags Fiesta Texas and UTSA. It is 5

# step 4

In [18]:
df4 = pd.read_csv('Airbnb_Texas_Rentals.csv')

city_lat_lon = df4.groupby('city').mean()[['latitude','longitude']]
#city_lat_lon.index = city_lat_lon.index.map(lambda x: nltk.stem.PorterStemmer().stem(nltk.word_tokenize(x)[0]))
#print(city_lat_lon)

In [ ]:
user_query_city = input('Enter city: ')
user_query_price = input('Enter your preferred daily price: ')
user_query_bedrooms = input('Enter your preferred number of bedrooms: ')
print(city, price, bedrooms)

In [ ]:
def score(doc_price ,doc_bedrooms, doc_latitude, doc_longitude):
    
    score = 0
    global user_query_city, user_query_price , user_query_bedrooms, city_lat_lon 
    
    city = user_query_city
    price = int(user_query_price .replace('$', ''))
    bedrooms = int(user_query_bedrooms)
    print(city)
    
    if city in city_lat_lon.index:
        lat_lon = list(city_lat_lon.loc[city])
        print(lat_lon)
        lat_lon_diff = [float(lat_lon[0]) - float(doc_latitude), float(lat_lon[1]) - float(doc_longitude)]
        
        print(lat_lon_diff)
        
        if (doc_price[1:].isdigit() == True) and (doc_bedrooms.isdigit() == True):
            if (abs(lat_lon_diff[0]) < 1.5) and (abs(lat_lon_diff[1]) < 1.5):
            
                #print(max(float(doc_price[1:]), price)
                score = (1/(1 + (abs(float(doc_price[1:]) - price))) * 0.6) + (1/(1 + (abs(int(doc_bedrooms) - bedrooms)))) * 0.4
            else:
                score = ((1/(1 + (abs(float(doc_price[1:]) - price))) * 0.6) + (1/(1 + (abs(int(doc_bedrooms) - bedrooms)))) * 0.4) * 0.1
                #print(max(float(doc_price[1:]), price)
                #score = (((max(float(doc_price[1:]), price)-min(float(doc_price[1:]), price)) /max(float(doc_price[1:]), price)) * 0.6) +((max(float(doc_bedrooms), bedrooms)-min(float(doc_bedrooms), bedrooms)) /max(float(doc_bedrooms), bedrooms)) * 0.4
    return score
    

In [37]:


step4_idx = {}
for j, doc in enumerate(intersect):
    with open(r'doc\{}'.format(doc), 'r',encoding="utf-8") as file:
        lines = file.readlines()
        if len(lines) > 0 :
            document = lines[0]
            cols = document.replace('\n',' ').split('\t')
            #print(cols)
            doc_price, doc_bedrooms, doc_latitude, doc_longitude = cols[0], cols[1], cols[5], cols[6]
            print(doc_price, doc_bedrooms)
            new_score = score(doc_price ,doc_bedrooms, doc_latitude, doc_longitude)
            print(new_score)
            step4_idx[doc] = new_score
            
save_dict_to_file(step4_idx,"step4_idx")
print(step4_idx)

Enter city, price per night, number of bedrooms: San Antonio 90 3
San Antonio 90 3
$80 Studio
San Antonio
[29.501488198655036, -98.50031031977231]
[-3.2538408393193627, -1.6570295646720155]
0
$45 1
San Antonio
[29.501488198655036, -98.50031031977231]
[-3.3512741378318616, -1.1230742553077135]
0
$59 1
San Antonio
[29.501488198655036, -98.50031031977231]
[-3.2363100159094635, -1.1292489435647042]
0
$40 1
San Antonio
[29.501488198655036, -98.50031031977231]
[-0.023933694726164134, 0.10625646058248606]
0.6
{'doc_121.tsv': 0, 'doc_221.tsv': 0, 'doc_397.tsv': 0, 'doc_497.tsv': 0.6}


In [21]:
user_query = input('Enter city, price per night, number of bedrooms: ').split(' ')
if len(user_query) > 3:
    user_query[0] = user_query[0] + ' ' + user_query[1] 
    del user_query[1]
user_query

Enter city, price per night, number of bedrooms: san antonio 40 2


['san antonio', '40', '2']

In [39]:
col_names = ["average_rate_per_night","bedrooms_count","city","date_of_listing","description","latitude","longitude","title","url", "similarity"]
df = pd.DataFrame(columns = col_names)# index = False?

heap = [(-value, key) for key, value in step4_idx.items()]
largest = heapq.nsmallest(5, heap)
largest = [(key, -value) for value, key in largest]
print(largest)

col_names = ["average_rate_per_night","bedrooms_count","city","date_of_listing","description","latitude","longitude","title","url", "score"]
df = pd.DataFrame(columns = col_names)# index = False?

for j, doc in enumerate(largest):
    with open(r'doc\{}'.format(doc[0]), 'r',encoding="utf-8") as file:
        
        csv_reader = csv.reader(file, delimiter = '\t')
        for line in csv_reader:
            for i, field in enumerate(line):
                df.loc[j, col_names[i]] = field
                df.loc[j, 'score'] = doc[1]
        

pd.set_option('max_colwidth',500)
display(df[['title', 'description', 'city', 'url', 'score']].style.set_table_styles([ {'selector': '.row_heading, .blank', 'props': [('display', 'none;')]}]))

[('doc_497.tsv', 0.6), ('doc_121.tsv', 0), ('doc_221.tsv', 0), ('doc_397.tsv', 0)]


,title,description,city,url,score
0,Private Rm/Bath @ the Gold House!,"This unique private guest room at the Gold House offers the amenities of home in a city full of things to do! The entire downstairs living room, kitchen, and dining area is open to guests. Come on down to SA town and experience Texan hospitality for yourself!\n\nPlease note: there is a second guest room at the Gold house, so if this listing is booked, do check with me to see if the second room is available! Thanks for your interest!\n\n\nCute House:\nIt's a private room upstairs with a full bed, small desk, walk-in closet, Ironing Board, and en suite bathroom. The window has a nice view of the back yard and wooded easement. The back yard is large with a little garden in the back. The kitchen was recently renovated with nice granite counter tops and new appliances. Also, there is a brand new running/hiking trail one mile away. Parking is safe and free in the form of my driveway.\n\nMore than Two People? No problem:\nThere is a second guestroom in the loft with a full bed that can be reserved via my other listing. http://airbnb.com/rooms/826725\n\nBreakfast/Snacks:\nThe living room/dining room/patio/kitchen are all fair game for guests. Breakfast is provided by way of oatmeal and any items found on the counter (Hot tea and coffee too). \n\nFour-Legged Friends: \nI do have two wonderful dogs (lab and husky), but don't worry they aren't allowed upstairs. They stay outside during the day, and sleep in their beds downstairs at night. They are both full of sugar - but if you have a fear of dogs then this place is not for you(URL HIDDEN)\nLocation:\nIt is literally right next to a brand new grocery store but you wouldn't know it from the back yard. It is 10-15 minutes from La Cantera - the gorgeous outdoor shopping mall. As well as Six Flags Fiesta Texas and UTSA. It is 5-10 minutes from the medical center area where all the hospitals are. It is 3 miles from IH-10, which is the main highway to get to downtown where the Riverwalk is (about 20 minutes by car). Airport - about 15-20 minutes away.\n\n\nThings to look up for your trip regardless of whether you choose the Gold House for your stay (we want you to have fun!):\n\nIn San Antonio...\n- The Riverwalk \n- Market Square\n- The Spanish Missions (Includes the Alamo of course...you can rent bikes to explore all of them!)\n- Six Flags Fiesta Texas Theme Park\n- Sea World Theme Park\n- Morgan's Wonderland (Special Needs Theme Park)\n- San Antonio Botanical Gardens\n- San Antonio Zoo\n\nWithin an Hour of San Antonio...\n- Schlitterbahn Water Park (Amazing rides, Bring Food, and Free Parking)\n- Natural Bridge Caverns (wondrous caves to stay cool)\n- Natural Bridge Wildlife Ranch (African Safari Texas Style)\n- Gruene, Texas (adorable hill country town with shopping and tubing on the river)\n\n\nThe entire downstairs!\n\nSafe neighborhood - within walking distance to a grocery store!\n\nBus stop is about a 10 minute walk from the front door.\n\nCheckout is more flexible if there are no guests arriving the same night you're leaving.",San Antonio,https://www.airbnb.com/rooms/1284796?location=Brazos%20River%2C%20TX,0.6
1,Oak Cliff Tiny House,"Warm open space guesthouse cultured as a meditation and massage studio for 20+ years. The space is filled with natural light with a vaulted, shiplap ceiling while offering a restful view onto our garden and gold fish pond. Small refrigerator, microwave, Keurig, toaster oven, towel warmer, hair dryer and bicycles to tour north Oak Cliff and the Bishop Arts District.\nHot tub included, yoga and massage services available, if desired.",Dallas,https://www.airbnb.com/rooms/8799838?location=Carrollton%2C%20TX,0
2,1 Bedroom in quiet neighborhood,"Private bedroom with queen size bed and private bathroom. Cable TV in room.\n\nLiving room with cable TV, dining room, kitchen, outdoor patio with grill and sitting area, free wifi, laundry room with washer and dryer\n\nBrad and I work Monday throu

# junk

In [6]:
with open('Airbnb_Texas_Rentals_new.csv','r',encoding="utf8") as csv_file:
    csv_reader=csv.reader(csv_file)
    
    with open('Parametric_search.tsv','w',encoding="utf8") as new_file: 
        csv_writer=csv.writer(new_file,delimiter='\t')
    
        for line in csv_reader:
               csv_writer.writerow(line[2:5]) 

with open('Parametric_search.tsv','r',encoding="utf8") as all_files:
    for i, row in enumerate(all_files):
        if i > 0 and i < 40:
            if row != '\n':
                with open('doc_par_search\doc_' + str(int((i-(i/2)))) + '.tsv','w',encoding="utf8") as new_files:
                    new_files.write(row) 

In [15]:
docpaths = [f for f in listdir("doc_par_search") if isfile(join("doc_par_search", f))]

user_query = input('Enter city, price per night, number of bedrooms: ').split(' ')
city, price, bedrooms = user_query[0], user_query[1], user_query[2]



par_search_doc_list = {}
for doc in docpaths:
    with open('doc_par_search\{}'.format(doc), 'r',encoding="utf8") as file:
        lines = file.readlines()
        if len(lines) > 0 :
            document = lines[0]
            cols = document.replace('\n', '').split('\t')
            #doc_price, doc_bedrooms, doc_city = cols[1], cols[2], cols[3]
            #print(doc_price, doc_bedrooms, doc_city)
            par_search_doc_list[doc] = list(cols)
            
save_dict_to_file(par_search_doc_list,"par_search_doc_list")

Enter city, price per night, number of bedrooms: a b c


In [ ]:
#par_search_inv_idx = {}
#with open('par_search_doc_list.txt', 'r', encoding="utf-8") as file:
    data = eval(file.read())
    for key,doc in documentlist.items():
        for w in doc:
            score = tfidf(w,doc,documentlist)
            w_index = (key,score)
            if vocabulary[w] not in new_inverted_index:
                temp = []
                temp.append(w_index)
                new_inverted_index[vocabulary[w]] = temp
            elif doc not in new_inverted_index[vocabulary[w]]:
                new_inverted_index[vocabulary[w]].append(w_index)    